# Skeletonization of FANC meshes

The `fanc` python package has a `skeletonize.py` module [here](https://github.com/htem/FANC_auto_recon/blob/main/fanc/skeletonize.py) containing functions that can help you generate skeletons from FANC neurons.

In [1]:
import fanc
example_neuron_soma_pt = [43232, 134024, 4218]
example_neuron_segid = fanc.lookup.segid_from_pt(example_neuron_soma_pt)
# As of July 23, 2023, the example neuron's segID is 648518346486614449

## Approach 1 (recommended): Use `pcg_skel`

In [2]:
# This will take ~1 minute to run
skeleton = fanc.skeletonize.get_pcg_skeleton(example_neuron_segid)
# For the record, the line above just calls pcg_skel.pcg_skeleton, so it's exactly the same as doing:
#import pcg_skel
#skeleton = pcg_skel.pcg_skeleton(example_neuron_segid)

Minishard Indices: 100%|████████████████████████████████████████████████████████████████████████████| 873/873 [00:38<00:00, 22.47it/s]


In [3]:
print(type(skeleton))  # Congrats, you now have a meshparty.skeleton.Skeleton object to do whatever with
print('Number of nodes', len(skeleton.vertices))

<class 'meshparty.skeleton.Skeleton'>
Number of nodes 988


In [4]:
# Let's look at the skeleton in neuroglancer overlaid on the segmentation
print(fanc.statebuilder.render_scene(neurons=example_neuron_segid,
                                     annotations=skeleton.vertices,
                                     annotation_units='nm'))

https://neuromancer-seung-import.appspot.com/?json_url=https://global.daf-apis.com/nglstate/api/v1/4787774739709952


Looks pretty good, but the points are quite spaced out. If you want a denser skeleton (more closely placed nodes), you could try Approach 2.

In [5]:
#Let's save it in .swc format for use in other programs
skeleton.export_to_swc(f'output/{example_neuron_segid}_pcg-skel.swc')

## Approach 2: Use meshparty/navis

This approach has slightly more complicated package dependencies, and I was only able to get it to work on a conda env with `conda install -c conda-forge pyembree` and then install everything else needed for the `fanc` package (either `cd ~/repos/htem/FANC_auto/recon; pip install -e .` to install from a local clone, or `pip install fanc-fly` to install from PyPI), plus finally `pip install embreex`.

- Output can be either `meshwork` or `navis`
- Default is `meshwork` as meshwork neuron objects have a great deal of utility, and can be converted rapidly to `navis` objects. 
- This uses meshparty skeletonization which is only appropriate for some applications. If the results don't look good, try `pcg_skel` (see Approach 1 above).

In [6]:
# This will take ~5 minutes to run
neuron_meshwork = fanc.skeletonize.skeletonize_neuron(
    example_neuron_segid,
    example_neuron_soma_pt,
    output='meshwork')

136


/Users/jasper/software/anaconda3/envs/fanc/lib/python3.10/site-packages/numpy/core/fromnumeric.py:86: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  return ufunc.reduce(obj, axis, dtype, out, **passkwargs)


Adding 2942570 new edges.
TIME MERGING: 163.879s


In [9]:
print(type(neuron_meshwork))  # Congrats, you now have a meshparty.Meshwork object
print('Number of nodes', len(neuron_meshwork.skeleton.vertices))

<class 'meshparty.meshwork.meshwork.Meshwork'>
Number of nodes 55795


This skeleton has 55795 nodes, wayyyy more than the 988 from `pcg_skel`. That means the node spacing is much denser, which gives a higher resolution skeleton approximation to the mesh.

In [10]:
# Let's look at the skeleton in neuroglancer overlaid on the segmentation.
# We can't build a scene with all the nodes (too many!) so we'll just take every 20th one.
print(fanc.statebuilder.render_scene(neurons=example_neuron_segid,
                                     annotations=neuron_meshwork.skeleton.vertices[::20, :],
                                     annotation_units='nm'))

https://neuromancer-seung-import.appspot.com/?json_url=https://global.daf-apis.com/nglstate/api/v1/5477088804995072


See https://github.com/ceesem/MeshworkTutorial and below for what can be done with meshwork objects.

In [11]:
# Example: Convert to navis for even more skeleton-based functionality: https://navis.readthedocs.io/en/latest/
neuron_navis = fanc.skeletonize.mp_to_navis(neuron_meshwork.skeleton)

In [12]:
# Example: Output as .swc file for use elsewhere.
neuron_navis.to_swc(f'output/{example_neuron_segid}_high-res-skeleton.swc')

## Compare Approach 1 and Approach 2

In [13]:
print(fanc.statebuilder.render_scene(
          neurons=example_neuron_segid,
          annotations=[{'type': 'points', 'name': 'pcg_skel', 'data': skeleton.vertices},
                       {'type': 'points', 'name': 'meshwork_skel', 'data': neuron_meshwork.skeleton.vertices[::20, :]}],
          annotation_units='nm'
))

https://neuromancer-seung-import.appspot.com/?json_url=https://global.daf-apis.com/nglstate/api/v1/6078644810153984


Even when using only 1 out of every 20 points in the meshwork skeleton, there points are still more dense than all the points from `pcg_skel`.

Hopefully this help you get a sense of which skeletonization method you want to use for your work:
- the much faster but much lower resolution `pcg_skel` via `fanc.skeletonize.get_pcg_skeleton()`, or
- the much slower but much higher resolution meshwork skeleton via `fanc.skeletonize.skeletonize_neuron()`